In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
%matplotlib inline

from pandas.io.json import json_normalize
import json
import random
import warnings
warnings.filterwarnings("ignore")

In [2]:
def load_df(csv_path='data/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    p = 0.1
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     skiprows=lambda i: i>0 and random.random() > p)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = ["{}.{}".format(column, subcolumn) for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print("Loaded {}. Shape: {}".format(os.path.basename(csv_path), df.shape))
    return df

In [3]:
data = load_df()

Loaded train.csv. Shape: (90391, 54)


In [4]:
data.shape

(90391, 54)

In [6]:
shops_or_not = lambda x : x.total

In [9]:
data['fullVisitorId'].nunique()

85767

In [25]:
data["totals.transactionRevenue"] = data["totals.transactionRevenue"].astype('float')
y_clf = (data['totals.transactionRevenue'].fillna(0) > 0).astype(np.uint8)
y_reg = data['totals.transactionRevenue'].fillna(0)

In [30]:
y_reg.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: totals.transactionRevenue, dtype: float64

In [31]:
y_clf.head()

0    0
1    0
2    0
3    0
4    0
Name: totals.transactionRevenue, dtype: uint8

In [34]:
y_reg.max()

5601400000.0

In [33]:
y_clf.mean()

0.01262293812437079

In [35]:
data['totals.transactionRevenue'].max()

5601400000.0

In [36]:
y_clf.max()

1

In [37]:
y_reg.max()

5601400000.0